## CRF What's Wrong with Participants? --II

In [1]:
from crf import *
from preprocess_data import get_all_data_train
import sys,os,pickle

import os, time, pprint

from features_generator import abstracts2features, get_genia_tags, sanity_check

from gensim.models import Word2Vec

/Users/Jing/anaconda/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Jing/anaconda/envs/tensorflow/lib/python2.7/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
tag = 'P'
eval_tags = [tag]
sentence=True

Get data

In [3]:
import os, pickle, sys
from collections import defaultdict
from preprocess_data import get_all_data_train, get_all_data_dev, get_all_data_test
from gensim.models import Word2Vec

# Directory for annotations
directory = 'PICO-annotations/batch5k'

# Suffixes for the generated files
tokens_suffix = '_tokens.txt'
genia_tags_suffix = '_genia.tag'

def get_genia_tags(data_set,sentence=False, tokens=False, tags=False):
    switcher = {
        'train': 'PICO-annotations/train_abstracts.txt',
        'dev': 'PICO-annotations/dev_abstracts.txt',
        'test': 'PICO-annotations/test_abstracts.txt', 
    }
    path = switcher[data_set];
    abstract_file = open(path, 'r')
    abstracts = abstract_file.readlines()
    abstracts = [x.strip() for x in abstracts]
    
    genia_tags = []
    sent_tokens =[]
    sent_tags = []
    
    for i,abstract_path in enumerate(abstracts):
        pickle_path = abstract_path[:-4] + genia_tags_suffix
        pickle_file = open(pickle_path, 'rb')
        abstract_genia_tags = pickle.load(pickle_file)
        if sentence:
            sentences_genia_tags = []
            start = 0
            end = 0
            abstract_len =len(abstract_genia_tags)
            tags_len = 0
            for j,sent in enumerate(tokens[i]):
                end = end+len(sent)
                sentences_genia_tags=abstract_genia_tags[start:end]
                
                
                genia_tags.append(sentences_genia_tags)
                sent_tokens.append(sent)
                sent_tags.append(tags[i][j])
                
                
                start = start+len(sent)
                tags_len = tags_len +len(sent)
        else:
            genia_tags.append(abstract_genia_tags)
            
    if sentence:
        return sent_tokens,sent_tags,genia_tags
    else:
        return genia_tags
    

In [4]:
# Get train data
train_tokens, train_tags = get_all_data_train(sentences=sentence)
train_tokens, train_tags,train_genia_tags = get_genia_tags('train',sentence=True,\
                                                           tokens=train_tokens,tags=train_tags)

In [5]:
# Get dev data
dev_tokens, dev_tags = get_all_data_dev(sentences=sentence)
dev_tokens, dev_tags,dev_genia_tags = get_genia_tags('dev',sentence=True,\
                                                     tokens=dev_tokens,
                                                     tags=dev_tags)


In [6]:
# Get test data
test_tokens, test_tags = get_all_data_test(sentences=sentence)
test_tokens, test_tags,test_genia_tags = get_genia_tags('test',sentence=True, tokens=test_tokens,
                                                     tags=test_tags)

In [7]:
feature_name='OneNegOneHot'
file_name='crf_results/{}'.format(feature_name)
f = open(file_name + '_results.txt','r')
result=pickle.load(f)
f.close()
result

{(0.0001, 0.0001): [0.52634933593511524,
  0.43697861157889167,
  0.47715768242587625],
 (0.0001, 0.001): [0.55854056838367594,
  0.44449206574046446,
  0.49487034840599514],
 (0.0001, 0.01): [0.60911415766341093,
  0.45603869820708953,
  0.52143931607158778],
 (0.0001, 0.1): [0.68131797232957758,
  0.46206041852235979,
  0.55063603276601092],
 (0.0001, 1): [0.75078030989922828, 0.46865655460635064, 0.57696627029307224],
 (0.001, 0.0001): [0.54642052201828384,
  0.43682664642721314,
  0.48550355155305897],
 (0.001, 0.001): [0.55921956396949013,
  0.44838277706991364,
  0.49753169823310722],
 (0.001, 0.01): [0.61454398670159172,
  0.45335926829725703,
  0.52171860996215913],
 (0.001, 0.1): [0.68264718102813748, 0.4635564626920633, 0.55213785235848623],
 (0.001, 1): [0.75690910229299357, 0.46447865196054972, 0.57558528966335065],
 (0.01, 0.0001): [0.59386039484298236,
  0.45955267946253187,
  0.51803234679204246],
 (0.01, 0.001): [0.60655275609492665,
  0.45538841751451431,
  0.519925419

In [ ]:
# result of the one neg one hot, no other features

dev:
P: (0.746385029770343, 0.4764274726269116, 0.5816072908036454)
train:
P: (0.8331695473726805, 0.5432444656632409, 0.6576723882242891)
test:
P: (0.7552387740555951, 0.4816801527411583, 0.5882091706450538)

In [71]:
#one hot one neg , no other features  -- sentence level
num_iters = 100
l1 = 0.1
l2 = 1

options_string = 'left_neighbors=1 right_neighbors=1 one_hot one_hot_neighbors'

w2v= False
features_name = 'OneNeigSent'

dev:
P: (0.7300115874855156, 0.48457153198805536, 0.58249258967177)
train:
P: (0.8660174963549261, 0.6268437319395934, 0.7272714021252714)
test:
P: (0.7339790367520234, 0.5029548140740068, 0.5968925334484247)


In [85]:
#one hot one neg , no other features, W2V  -- sentence level
num_iters = 100
l1 = 0.1
l2 = 1

options_string = 'left_neighbors=1 right_neighbors=1 one_hot one_hot_neighbors w2v w2v_neighbors w2v_size=30'

w2v= pubmed_w2v
features_name = 'OneNeigSentw2v'

dev:
P: (0.7343526039178213, 0.4867885259252556, 0.5854760155633554)
train:
P: (0.874067643463317, 0.6198331532527578, 0.7253173768901108)
test:
P: (0.7331300543550312, 0.5027729793617601, 0.5964836587207422)


In [18]:
options_string = 'left_neighbors=3 right_neighbors=3 one_hot one_hot_neighbors \
inside_paren pos chunk iob named_entity \
inside_paren_neighbors pos_neighbors chunk_neighbors iob_neighbors named_entity_neighbors \
chunk_end chunk_end_neighbors same_chunk_neighbors \
w2v_model=pubmed_wiki w2v w2v_neighbors w2v_size=30 \
cosine_simil cosine_simil_neighbors \
isupper isupper_neighbors istitle istitle_neighbors'

w2v= pubmed_wiki_w2v
features_name = '3NeigSentBest8'

# Set options
num_iters = 200
l1 = 0.1
l2 = 1
file_name = 'crf_results/{}'.format(features_name)

dev:
P: (0.7699750112129172, 0.543706451904805, 0.6373544777108914)
train:
P: (0.9433982501570568, 0.7735382063974672, 0.8500659270867131)
test:
P: (0.7835024720270621, 0.5475043185744158, 0.6445812148782446)


In [73]:
pubmed_w2v_name = 'PubMed-w2v.bin'

# Load pubmed word2vec model (1-2 min)
pubmed_w2v = Word2Vec.load_word2vec_format(pubmed_w2v_name, binary=True)

In [9]:
pubmed_wiki_w2v_name = 'wikipedia-pubmed-and-PMC-w2v.bin'
pubmed_wiki_w2v = Word2Vec.load_word2vec_format(pubmed_wiki_w2v_name, binary=True)

In [10]:
# Set options
big_options_string = 'left_neighbors=3 right_neighbors=3 inside_paren pos chunk iob named_entity \
inside_paren_neighbors pos_neighbors chunk_neighbors iob_neighbors named_entity_neighbors \
chunk_end chunk_end_neighbors same_chunk_neighbors \
one_hot one_hot_neighbors w2v_model=pubmed w2v w2v_neighbors w2v_size=10 cosine_simil cosine_simil_neighbors \
isupper isupper_neighbors istitle istitle_neighbors'

options_string = 'left_neighbors=3 right_neighbors=3 one_hot one_hot_neighbors \
inside_paren pos chunk iob named_entity \
inside_paren_neighbors pos_neighbors chunk_neighbors iob_neighbors named_entity_neighbors \
chunk_end chunk_end_neighbors same_chunk_neighbors \
w2v_model=pubmed_wiki w2v w2v_neighbors w2v_size=30 \
cosine_simil cosine_simil_neighbors \
isupper isupper_neighbors istitle istitle_neighbors'

w2v= pubmed_wiki_w2v
features_name = '3NeigSentBest8'

In [11]:
# Compute features for train
train_features = abstracts2features(train_tokens, train_genia_tags, w2v=w2v, options_string=options_string)

38481: ['The', 'shorter', 'and']


In [12]:
# Compute features for dev
dev_features = abstracts2features(dev_tokens, dev_genia_tags, w2v=w2v, options_string=options_string)

10999: ['aureus', 'was', 'greater']


In [ ]:
# Compute features for test
test_features = abstracts2features(test_tokens, test_genia_tags, w2v=w2v, options_string=options_string)

5651: ['This', 'paper', 'outlines']


In [ ]:
# For debug
sanity_check(train_features)

Grid search

In [ ]:
# # # Run grid search
# grid_file_name = 'crf_results/{}_grid'.format(features_name)
# num_iters = 100
# l1_list = [ 0.0001, 0.001, 0.01, 0.1, 1]
# l2_list = [ 0.0001, 0.001, 0.01, 0.1, 1]

# start_time = time.time()
# grid_search_result = grid_search(train_features, train_tags, dev_features, dev_tags,\
#                                  num_iters, l1_list, l2_list, eval_tags, file_name=grid_file_name, save=True)
# print("--- %s seconds ---" % (time.time() - start_time))

# os.remove(grid_file_name + '.model')

In [ ]:
# # Sort result
# sorted_result = sort_by_metric(grid_search_result, tag, metric='f1')
# print_result(sorted_result)

Run CRF

In [ ]:
# Set options
num_iters = 200
l1 = 1
l2 = 0.01
file_name = 'crf_results/{}'.format(features_name)

In [ ]:
# Run CRF
start_time = time.time()
crf_result = get_crf_results(train_features, train_tags, dev_features, dev_tags, num_iters, l1, l2, eval_tags,
                             file_name=file_name, save=True)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# Print result
print_result(crf_result)

Get model

In [ ]:
# Get model from file

# features_name = 'Feat7OneNeg'
# file_name = 'crf_results/{}'.format(features_name)
tagger = get_tagger(file_name)

In [ ]:
# For debug
print_model_info(tagger)

Predict tags

In [ ]:
# Predict dev tags
pred_dev_tags = predict_tags(tagger, dev_features)

# Evaluate dev tags
dev_result = evaluate_prediction(pred_dev_tags, dev_tags, eval_tags)
print 'dev:'
print_result(dev_result)
sys.stdout.flush()

# Predict train tags
pred_train_tags = predict_tags(tagger, train_features)

# Evaluate train tags
train_result = evaluate_prediction(pred_train_tags, train_tags, eval_tags)
print 'train:'
print_result(train_result)
sys.stdout.flush()

# Predict test tags
pred_test_tags = predict_tags(tagger, test_features)

# Evaluate test tags
test_result = evaluate_prediction(pred_test_tags, test_tags, eval_tags)
print 'test:'
print_result(test_result)
sys.stdout.flush()

K-fold evaluation

In [25]:
# Run K-fold
kfold_file_name = 'crf_results/{}_kfold'.format(features_name)

start_time = time.time()
kfold_result = get_kfold_results(train_features, train_tags, num_iters, l1, l2, eval_tags,\
                                 file_name=kfold_file_name, save=True)
print("--- %s seconds ---" % (time.time() - start_time))

os.remove(kfold_file_name + '.model')

On fold 0
Adding data...
Training model...
Done!
On fold 1
Adding data...
Training model...
Done!
On fold 2
Adding data...
Training model...
Done!
On fold 3
Adding data...
Training model...
Done!
On fold 4
Adding data...
Training model...
Done!
--- 162.671914816 seconds ---


In [26]:
# Print all results
print_result(kfold_result)

Fold 0
P: (0.743448275862069, 0.4230492510955589, 0.5392471549460169)
Fold 1
P: (0.7338930105427567, 0.4609724691887915, 0.5662636990170602)
Fold 2
P: (0.712115031613298, 0.4341311781162574, 0.5394152408172724)
Fold 3
P: (0.7171599922615592, 0.4529569892473118, 0.5552310342245188)
Fold 4
P: (0.7343669781291714, 0.46014282956958114, 0.565778023890515)
Average
P: (0.72819665768177089, 0.44625054344350013, 0.5531870305790767)


In [41]:
def print_with_spaces(l, spaces):
    # This pads strings to be of space length and aligned left
    formatter = lambda space: '{:' + str(space) + '}'
    
    for sublist in l:
        print ''.join([formatter(space).format(string) for string, space in zip(sublist, spaces)])

In [55]:
i = 2
print_with_spaces(zip(dev_tokens[i], dev_tags[i], pred_dev_tags[i]), [25, 5, 5])

Netilmicin               None None 
in                       P    None 
the                      P    None 
neonate                  P    None 
:                        None None 
pharmacokinetic          None None 
analysis                 None None 
and                      None None 
influence                None None 
of                       None None 
parenteral               None None 
nutrition                None None 
.                        None None 
OBJECTIVE                None None 
The                      None None 
aim                      None None 
of                       None None 
this                     None None 
study                    None None 
was                      None None 
to                       None None 
investigate              None None 
the                      None None 
impact                   None None 
of                       None None 
parenteral               None None 
nutrition                None None 
on                       Non